In [1]:
import os
import subprocess
import time
from mininet.net import Mininet
from mininet.topo import Topo
from mininet.node import Switch
from mininet.link import TCLink
from mininet.cli import CLI
from mininet.log import setLogLevel, info, error

class P4Switch(Switch):
    def __init__(self, name, json_file, thrift_port, device_id, **params):
        Switch.__init__(self, name, **params)
        self.json_file = json_file
        self.thrift_port = thrift_port
        self.device_id = device_id
        self.proc = None

    def start(self, controllers):
        cmd = [
            'simple_switch',
            '--thrift-port', str(self.thrift_port),
            '--device-id', str(self.device_id),
            '--nanolog', f'ipc:///tmp/bm-{self.device_id}-log.ipc',
            '--log-console',
            self.json_file
        ]
        for idx, intf in enumerate(self.intfList()):
            if intf.name != "lo":
                cmd.extend(['-i', f"{idx+1}@{intf.name}"])
        info(f"Starting P4 switch {self.name}: {' '.join(cmd)}\n")
        self.proc = subprocess.Popen(
            cmd,
            stdout=open(f'{self.name}.log', 'w'),
            stderr=subprocess.STDOUT
        )
        time.sleep(1)

    def stop(self):
        if self.proc:
            self.proc.terminate()
            self.proc.wait()

class FastRerouteTopo(Topo):
    def __init__(self, json_file):
        Topo.__init__(self)
        switches = []
        for i in range(1, 4):
            sw = self.addSwitch(
                f's{i}',
                cls=P4Switch,
                json_file=json_file,
                thrift_port=9090 + (i-1),
                device_id=i
            )
            switches.append(sw)

        hosts = []
        for i in range(1, 4):
            host = self.addHost(
                f'h{i}',
                ip=f'10.0.{i}.1/24',
                mac=f'00:00:00:00:0{i}:01',
                defaultRoute=f'via 10.0.{i}.254'
            )
            hosts.append(host)
            self.addLink(host, switches[i-1], port1=0, port2=2, cls=TCLink, bw=100, delay='1ms')

        # Primary links
        self.addLink(switches[0], switches[1], port1=3, port2=3, cls=TCLink, bw=50, delay='1ms')  # s1-s2
        self.addLink(switches[1], switches[2], port1=4, port2=3, cls=TCLink, bw=50, delay='1ms')  # s2-s3
        self.addLink(switches[2], switches[0], port1=4, port2=4, cls=TCLink, bw=50, delay='1ms')  # s3-s1

        # Backup links
        self.addLink(switches[0], switches[1], port1=5, port2=5, cls=TCLink, bw=30, delay='10ms')  # s1-s2
        self.addLink(switches[1], switches[2], port1=6, port2=5, cls=TCLink, bw=30, delay='10ms')  # s2-s3
        self.addLink(switches[2], switches[0], port1=6, port2=6, cls=TCLink, bw=30, delay='10ms')  # s3-s1

def configure_switch(sw):
    thrift_port = sw.thrift_port
    config = [
        'register_write link_status 2 1',  # Port to host
        'register_write link_status 3 1',  # Primary port
        'register_write link_status 4 1',  # Primary port
        'register_write link_status 5 1',  # Backup port
        'register_write link_status 6 1',  # Backup port
    ]

    if sw.name == 's1':
        config += [
            'table_add arp_table generate_arp_reply 10.0.1.254 => 00:00:00:00:01:01 10.0.1.254',
            'table_add ipv4_lpm set_routes 10.0.1.1/32 => 2 00:00:00:00:01:01 0 00:00:00:00:00:00 00:00:00:00:01:01',
            'table_add ipv4_lpm set_routes 10.0.2.1/32 => 3 00:00:00:00:02:02 5 00:00:00:00:02:04 00:00:00:00:01:02',
            'table_add ipv4_lpm set_routes 10.0.3.1/32 => 4 00:00:00:00:03:03 6 00:00:00:00:03:06 00:00:00:00:01:03',
            'table_add backup_routes set_backup_nhop 10.0.2.1/32 => 00:00:00:00:02:04 5 00:00:00:00:01:04',
            'table_add backup_routes set_backup_nhop 10.0.3.1/32 => 00:00:00:00:03:06 6 00:00:00:00:01:06',
        ]
    elif sw.name == 's2':
        config += [
            'table_add arp_table generate_arp_reply 10.0.2.254 => 00:00:00:00:02:01 10.0.2.254',
            'table_add ipv4_lpm set_routes 10.0.2.1/32 => 2 00:00:00:00:02:01 0 00:00:00:00:00:00 00:00:00:00:02:01',
            'table_add ipv4_lpm set_routes 10.0.3.1/32 => 4 00:00:00:00:03:03 6 00:00:00:00:03:05 00:00:00:00:02:03',
            'table_add ipv4_lpm set_routes 10.0.1.1/32 => 3 00:00:00:00:01:02 5 00:00:00:00:01:04 00:00:00:00:02:02',
            'table_add backup_routes set_backup_nhop 10.0.3.1/32 => 00:00:00:00:03:05 6 00:00:00:00:02:05',
            'table_add backup_routes set_backup_nhop 10.0.1.1/32 => 00:00:00:00:01:04 5 00:00:00:00:02:04',
        ]
    elif sw.name == 's3':
        config += [
            'table_add arp_table generate_arp_reply 10.0.3.254 => 00:00:00:00:03:01 10.0.3.254',
            'table_add ipv4_lpm set_routes 10.0.3.1/32 => 2 00:00:00:00:03:01 0 00:00:00:00:00:00 00:00:00:00:03:01',
            'table_add ipv4_lpm set_routes 10.0.1.1/32 => 4 00:00:00:00:01:03 6 00:00:00:00:01:06 00:00:00:00:03:02',
            'table_add ipv4_lpm set_routes 10.0.2.1/32 => 3 00:00:00:00:02:03 5 00:00:00:00:02:05 00:00:00:00:03:03',
            'table_add backup_routes set_backup_nhop 10.0.1.1/32 => 00:00:00:00:01:06 6 00:00:00:00:03:06',
            'table_add backup_routes set_backup_nhop 10.0.2.1/32 => 00:00:00:00:02:05 5 00:00:00:00:03:05',
        ]

    try:
        subprocess.run(
            ['simple_switch_CLI', '--thrift-port', str(thrift_port)],
            input='\n'.join(config),
            text=True,
            check=True,
            timeout=10
        )
        info(f"Configured switch {sw.name} successfully\n")
    except subprocess.SubprocessError as e:
        error(f"Failed to configure {sw.name}: {str(e)}\n")

def read_registers(net, switch_name, thrift_port, output_file):
    info(f"\n*** {switch_name} Registers:\n")
    with open(output_file, 'w') as f:
        subprocess.run(f"echo 'register_read link_status' | simple_switch_CLI --thrift-port {thrift_port}", shell=True, stdout=f)
        subprocess.run(f"echo 'register_read backup_counter' | simple_switch_CLI --thrift-port {thrift_port}", shell=True, stdout=f)
        subprocess.run(f"echo 'register_read drop_counter' | simple_switch_CLI --thrift-port {thrift_port}", shell=True, stdout=f)
        subprocess.run(f"echo 'register_read active_port' | simple_switch_CLI --thrift-port {thrift_port}", shell=True, stdout=f)
        subprocess.run(f"echo 'register_read h1_to_h2_active_port' | simple_switch_CLI --thrift-port {thrift_port}", shell=True, stdout=f)
    with open(output_file, 'r') as f:
        info(f.read())

def log_registers_periodically(net, switch_name, thrift_port, base_filename):
    for i in range(20):  # Log every 2 seconds for 40 seconds
        time.sleep(2)
        timestamp = i * 2
        output_file = f"{base_filename}_t{timestamp}.txt"
        info(f"*** Logging {switch_name} Registers at {timestamp} seconds\n")
        with open(output_file, 'w') as f:
            subprocess.run(f"echo 'register_read h1_to_h2_active_port' | simple_switch_CLI --thrift-port {thrift_port}", shell=True, stdout=f)
        with open(output_file, 'r') as f:
            info(f.read())

def compile_p4_program(p4_file):
    json_file = p4_file.replace('.p4', '.json')
    try:
        result = subprocess.run(
            ['p4c-bm2-ss', '--std', 'p4-16', '-o', json_file, p4_file],
            check=True,
            capture_output=True,
            text=True
        )
        info("P4 compilation succeeded:\n" + result.stderr + "\n")
        return json_file
    except subprocess.CalledProcessError as e:
        error("P4 compilation failed:\n" + e.stderr + "\n")
        return None

def execute_switch_cmd(net, switch_name, thrift_port, cmd):
    try:
        result = subprocess.run(
            ['simple_switch_CLI', '--thrift-port', str(thrift_port)],
            input=cmd,
            text=True,
            capture_output=True,
            check=True,
            timeout=5
        )
        info(f"Executed '{cmd}' on {switch_name}: {result.stdout}\n")
        return True
    except subprocess.SubprocessError as e:
        error(f"Failed to execute '{cmd}' on {switch_name}: {e.stderr}\n")
        return False

def verify_link_delays(net):
    info("*** Verifying link delays\n")
    s1 = net.get('s1')
    s2 = net.get('s2')
    # Verify primary link (should be 1ms)
    info("Primary link s1-eth3 delay:\n")
    s1.cmdPrint('tc qdisc show dev s1-eth3')
    info("Primary link s2-eth3 delay:\n")
    s2.cmdPrint('tc qdisc show dev s2-eth3')
    # Verify backup link (should be 10ms)
    info("Backup link s1-eth5 delay:\n")
    s1.cmdPrint('tc qdisc show dev s1-eth5')
    info("Backup link s2-eth5 delay:\n")
    s2.cmdPrint('tc qdisc show dev s2-eth5')
    # Ensure backup link delay is set correctly
    s1.cmd('tc qdisc replace dev s1-eth5 root netem delay 10ms')
    s2.cmd('tc qdisc replace dev s2-eth5 root netem delay 10ms')
    info("Backup link delays set to 10ms\n")

def main():
    setLogLevel('info')
    os.system('pkill -f "simple_switch|bmv2"')
    
    json_file = compile_p4_program("rerouteDeep.p4")
    if not json_file:
        error("Failed to compile P4 program\n")
        return

    net = Mininet(
        topo=FastRerouteTopo(json_file),
        controller=None,
        link=TCLink,
        autoSetMacs=True
    )
    net.start()

    try:
        # Configure switches
        for sw in net.switches:
            configure_switch(sw)
            time.sleep(0.5)

        # Ensure hosts respond to pings
        for host in net.hosts:
            host.cmd('sysctl -w net.ipv4.icmp_echo_ignore_all=0 >/dev/null')

        # Preconfigure ARP to avoid delays
        h1 = net.get('h1')
        h2 = net.get('h2')
        h3 = net.get('h3')
        h1.cmd('arp -s 10.0.2.254 00:00:00:00:02:01')
        h2.cmd('arp -s 10.0.1.254 00:00:00:00:01:01')
        h3.cmd('arp -s 10.0.1.254 00:00:00:00:01:01')
        h3.cmd('arp -s 10.0.2.254 00:00:00:00:02:01')
        info("*** Preconfigured ARP entries on hosts\n")

        # Verify link delays
        verify_link_delays(net)

        # Initial state
        info("*** Initial Register State\n")
        read_registers(net, 's1', 9090, 's1_initial_regs.txt')
        read_registers(net, 's2', 9091, 's2_initial_regs.txt')
        read_registers(net, 's3', 9092, 's3_initial_regs.txt')

        # Start continuous ping with timestamps in background
        info("*** Starting continuous ping from h1 to h2\n")
        h1.cmd('ping -D -i 0.1 -c 400 10.0.2.1 > h1_ping.log 2>&1 &')  # 400 pings to cover 40s
        time.sleep(2)  # Allow ping to start

        # Start periodic logging of h1_to_h2_active_port on s1
        info("*** Starting periodic logging of h1_to_h2_active_port on s1\n")
        subprocess.Popen(['python3', '-c', '''
import subprocess
import time
for i in range(20):  # Log every 2 seconds for 40 seconds
    timestamp = i * 2
    output_file = f"s1_periodic_t{timestamp}.txt"
    with open(output_file, 'w') as f:
        subprocess.run("echo 'register_read h1_to_h2_active_port' | simple_switch_CLI --thrift-port 9090", shell=True, stdout=f)
    time.sleep(2)
'''], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Start background traffic to create congestion
        info("*** Starting background traffic from h3 to h2\n")
        h2.cmd('iperf -s -u -p 5001 > h2_iperf_server.log 2>&1 &')  # Background server
        time.sleep(1)  # Allow server to start
        iperf_cmd = h3.cmd('iperf -c 10.0.2.1 -u -p 5001 -b 25M -t 45 -i 1 > h3_iperf_client.log 2>&1 &')
        info(f"iperf client output: {iperf_cmd}")
        time.sleep(12)  # Allow iperf to generate initial data (12s to reach 14s total)

        # Read registers during congestion (pre-failure, at ~14s)
        info("*** Register State During Congestion (Pre-Failure)\n")
        read_registers(net, 's1', 9090, 's1_congestion_pre_failure.txt')
        read_registers(net, 's2', 9091, 's2_congestion_pre_failure.txt')
        read_registers(net, 's3', 9092, 's3_congestion_pre_failure.txt')

        # Simulate link failure between s1 and s2 (primary link) at 15s
        info("\n*** Simulating link failure between s1 and s2 (primary link)\n")
        time.sleep(1)  # Ensure we hit 15s
        s1 = net.get('s1')
        s2 = net.get('s2')
        s1_s2_links = net.linksBetween(s1, s2)
        info(f"Links between s1 and s2: {s1_s2_links}\n")
        primary_link_found = False
        for link in s1_s2_links:
            info(f"Checking link: {link.intf1.name} <-> {link.intf2.name}\n")
            if link.intf1.name == 's1-eth3' and link.intf2.name == 's2-eth3':
                info(f"Disabling primary link: {link}\n")
                try:
                    link.stop()
                    time.sleep(0.5)  # Ensure link is fully down
                except AttributeError as e:
                    error(f"Failed to stop link {link}: {e}\n")
                primary_link_found = True
                if (execute_switch_cmd(net, 's1', 9090, 'register_write link_status 3 0') and
                    execute_switch_cmd(net, 's2', 9091, 'register_write link_status 3 0')):
                    info("Link failure commands executed successfully\n")
                else:
                    error("Link failure commands failed\n")
                break
        if not primary_link_found:
            error("Primary link s1-eth3 <-> s2-eth3 not found! Listing all links:\n")
            for link in net.links:
                info(f"Link: {link.intf1.name} <-> {link.intf2.name}\n")
        time.sleep(25)  # Capture post-failure data until ~40s

        # Post-failure state
        info("*** Post-Failure Register State\n")
        read_registers(net, 's1', 9090, 's1_post_regs.txt')
        read_registers(net, 's2', 9091, 's2_post_regs.txt')
        read_registers(net, 's3', 9092, 's3_post_regs.txt')

        info("*** Testing backup path\n")
        net.pingAll()

        # Display logs
        info("*** Full h1 ping log:\n")
        with open('h1_ping.log', 'r') as f:
            info(f.read())
        info("*** h3 to h2 iperf client log:\n")
        with open('h3_iperf_client.log', 'r') as f:
            info(f.read())
        info("*** h2 iperf server log:\n")
        with open('h2_iperf_server.log', 'r') as f:
            info(f.read())

        CLI(net)
    finally:
        h1.cmd('pkill -f "ping.*10.0.2.1"')
        h2.cmd('pkill -f "iperf.*5001"')
        h3.cmd('pkill -f "iperf.*10.0.2.1"')
        time.sleep(2)  # Ensure processes terminate
        try:
            net.stop()
        except AttributeError as e:
            error(f"Error during net.stop(): {e}\n")
        os.system('pkill -f "simple_switch|bmv2"')

if __name__ == '__main__':
    main()

P4 compilation failed:
rerouteDeep.p4(81): [--Werror=type-error] error: drop_counter.read: extern register does not have method matching this call
...ter.write(standard_metadata.ingress_port, drop_counter.read(standard_metadata.ingress_port) + 1);
                                             ^^^^^^^^^^^^^^^^^
/usr/local/share/p4c/p4include/v1model.p4(292)
extern register<T>
       ^^^^^^^^
rerouteDeep.p4(89): [--Werror=type-error] error: 'active_port.write(standard_metadata.ingress_port, port)'
        active_port.write(standard_metadata.ingress_port, port);
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  ---- Actual error:
  Cannot cast implicitly type 'bit<9>' to type 'bit<32>'
  ---- Originating from:
  rerouteDeep.p4(89): Type of argument 'standard_metadata.ingress_port' (bit<9>) does not match type of parameter 'index' (bit<32>)
          active_port.write(standard_metadata.ingress_port, port);
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  /usr/loc

In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np

# Parse ping log (already fixed to correctly detect packet loss)
def parse_ping_log(file_path):
    timestamps, seqs, times = [], [], []
    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()
            expected_seq = 1
            for line in lines:
                if 'bytes from' in line:
                    timestamp = float(re.search(r'\[(.*?)\]', line).group(1))
                    seq = int(re.search(r'icmp_seq=(\d+)', line).group(1))
                    time = float(re.search(r'time=(\d+\.\d+)', line).group(1))
                    while expected_seq < seq:
                        if timestamps:
                            last_timestamp = timestamps[-1]
                            estimated_timestamp = last_timestamp + 0.1 * (expected_seq - seqs[-1])
                        else:
                            estimated_timestamp = timestamp
                        timestamps.append(estimated_timestamp)
                        seqs.append(expected_seq)
                        times.append(None)
                        expected_seq += 1
                    timestamps.append(timestamp)
                    seqs.append(seq)
                    times.append(time)
                    expected_seq = seq + 1
                elif 'timeout' in line or 'unreachable' in line:
                    timestamp = float(re.search(r'\[(.*?)\]', line).group(1))
                    seq = int(re.search(r'icmp_seq=(\d+)', line).group(1))
                    while expected_seq < seq:
                        last_timestamp = timestamps[-1] if timestamps else timestamp
                        estimated_timestamp = last_timestamp + 0.1 * (expected_seq - seqs[-1]) if seqs else timestamp
                        timestamps.append(estimated_timestamp)
                        seqs.append(expected_seq)
                        times.append(None)
                        expected_seq += 1
                    timestamps.append(timestamp)
                    seqs.append(seq)
                    times.append(None)
                    expected_seq = seq + 1
        if not timestamps:
            raise ValueError("No valid ping data found in the log file.")
        timestamps = [t - timestamps[0] for t in timestamps]
    except FileNotFoundError:
        print(f"Error: {file_path} not found. Using dummy ping data.")
        timestamps = list(range(0, 40, 1))
        seqs = list(range(1, 41))
        times = [7.5 if t < 15 else 25.0 for t in timestamps]
    return pd.DataFrame({'timestamp': timestamps, 'seq': seqs, 'time_ms': times})

# Parse register file (unchanged)
def parse_registers(file_path):
    link_status, backup_counter, h1_to_h2_active_port = None, None, None
    try:
        with open(file_path, 'r') as f:
            for line in f:
                if 'link_status=' in line:
                    link_status = [int(x) for x in line.split('=')[1].split(',')][:7]
                if 'backup_counter=' in line:
                    backup_counter = int(line.split('=')[1].strip())
                if 'h1_to_h2_active_port=' in line:
                    h1_to_h2_active_port = int(line.split('=')[1].strip())
        if link_status is None or backup_counter is None:
            raise ValueError(f"Missing required register data in {file_path}.")
        if h1_to_h2_active_port is None:
            print(f"Warning: h1_to_h2_active_port not found in {file_path}. Defaulting to 0.")
            h1_to_h2_active_port = 0
    except FileNotFoundError:
        print(f"Error: {file_path} not found. Using dummy register data.")
        link_status = [0, 0, 1, 1, 1, 1, 1]
        backup_counter = 0
        h1_to_h2_active_port = 0
    return {
        'link_status': link_status,
        'backup_counter': backup_counter,
        'h1_to_h2_active_port': h1_to_h2_active_port
    }

# Parse periodic register logs (0.5-second intervals)
def parse_periodic_registers(base_filename, num_logs=80):
    timestamps = []
    active_ports = []
    for i in range(num_logs):
        timestamp = i * 0.5
        file_path = f"{base_filename}_t{timestamp}.txt"
        try:
            with open(file_path, 'r') as f:
                for line in f:
                    if 'h1_to_h2_active_port=' in line:
                        active_port = int(line.split('=')[1].strip())
                        timestamps.append(timestamp)
                        active_ports.append(active_port)
                        break
                else:
                    print(f"Warning: h1_to_h2_active_port not found in {file_path}. Skipping.")
        except FileNotFoundError:
            print(f"Warning: {file_path} not found. Skipping.")
    if not timestamps:
        print("Error: No periodic register data found. Using dummy data.")
        timestamps = list(np.arange(0, 40, 0.5))
        active_ports = [3 if t < 15 else 5 for t in timestamps]
    return pd.DataFrame({'timestamp': timestamps, 'active_port': active_ports})

# Parse iperf log for throughput (unchanged)
def parse_iperf_log(file_path):
    throughput_data = {'timestamp': [], 'throughput': []}
    try:
        with open(file_path, 'r') as f:
            content = f.read()
            print(f"Raw iperf log content:\n{content}")
            for line in content.splitlines():
                match = re.search(r'\[\s*\d+\]\s*(\d+\.\d+-\d+\.\d+)\s+sec\s+\d+\.\d+\s+MBytes\s+(\d+\.\d+)\s+Mbits/sec', line)
                if match:
                    start_time = float(match.group(1).split('-')[0])
                    throughput_val = float(match.group(2))
                    throughput_data['timestamp'].append(start_time)
                    throughput_data['throughput'].append(throughput_val)
                else:
                    match_alt = re.search(r'(\d+\.\d+)\s+Mbits/sec', line)
                    if match_alt:
                        throughput_data['throughput'].append(float(match_alt.group(1)))
                        throughput_data['timestamp'].append(len(throughput_data['timestamp']) * 1.0)
        if not throughput_data['throughput']:
            print("No throughput data found in log. Using dummy data for debugging.")
            throughput_data['timestamp'] = [0, 5, 10, 15, 20, 25, 30]
            throughput_data['throughput'] = [25, 24, 23, 15, 14, 13, 12]
    except FileNotFoundError:
        print(f"Error: {file_path} not found. Using dummy data.")
        throughput_data['timestamp'] = [0, 5, 10, 15, 20, 25, 30]
        throughput_data['throughput'] = [25, 24, 23, 15, 14, 13, 12]
    return pd.DataFrame(throughput_data)

# Calculate packet loss rate over time
def calculate_packet_loss_rate(ping_data, window_size=1.0):
    ping_data = ping_data.sort_values('seq')
    timestamps = ping_data['timestamp'].values
    seqs = ping_data['seq'].values
    times = ping_data['time_ms'].values

    expected_seqs = np.arange(seqs[0], seqs[-1] + 1)
    actual_seqs = set(seqs)
    missing_seqs = [seq for seq in expected_seqs if seq not in actual_seqs]

    loss_timestamps = []
    for missing_seq in missing_seqs:
        before_idx = np.where(seqs < missing_seq)[0][-1] if np.any(seqs < missing_seq) else None
        after_idx = np.where(seqs > missing_seq)[0][0] if np.any(seqs > missing_seq) else None
        if before_idx is not None and after_idx is not None:
            t_before = timestamps[before_idx]
            t_after = timestamps[after_idx]
            s_before = seqs[before_idx]
            s_after = seqs[after_idx]
            t_missing = t_before + (t_after - t_before) * (missing_seq - s_before) / (s_after - s_before)
            loss_timestamps.append(t_missing)
        elif before_idx is not None:
            t_before = timestamps[before_idx]
            loss_timestamps.append(t_before + 0.1)
        elif after_idx is not None:
            t_after = timestamps[after_idx]
            loss_timestamps.append(t_after - 0.1)

    max_time = timestamps[-1]
    bins = np.arange(0, max_time + window_size, window_size)
    sent_counts, _ = np.histogram(timestamps, bins=bins)
    loss_counts, _ = np.histogram(loss_timestamps, bins=bins)
    bin_centers = (bins[:-1] + bins[1:]) / 2

    total_packets = sent_counts + loss_counts
    loss_rates = np.where(total_packets > 0, loss_counts / total_packets, 0) * 100
    return bin_centers, loss_rates, sent_counts, loss_counts

# Load data
ping_data = parse_ping_log('h1_ping.log')
s1_initial = parse_registers('s1_initial_regs.txt')
s1_congestion = parse_registers('s1_congestion_pre_failure.txt')
s1_post = parse_registers('s1_post_regs.txt')
periodic_data = parse_periodic_registers('s1_periodic', num_logs=80)
iperf_data = parse_iperf_log('h3_iperf_client.log')

# Failure time and reroute time
failure_time = 15.0
reroute_time = 16.0  # Based on the packet loss rate spike ending at 16 seconds

# Calculate packet loss rate
window_size = 1.0
loss_timestamps, loss_rates, sent_counts, loss_counts = calculate_packet_loss_rate(ping_data, window_size)

# 1. Combined Latency, Port Usage, and Packet Loss Rate Plot
fig, ax1 = plt.subplots(figsize=(14, 8))

# Latency Plot with traffic flow indication (left y-axis)
pre_failure = ping_data[ping_data['timestamp'] < failure_time]
transition = ping_data[(ping_data['timestamp'] >= failure_time) & (ping_data['timestamp'] < reroute_time)]
post_failure = ping_data[ping_data['timestamp'] >= reroute_time]
ax1.plot(pre_failure['timestamp'], pre_failure['time_ms'], 'b-', label='Ping Latency (Primary Path)', linewidth=1.5)
ax1.plot(transition['timestamp'], transition['time_ms'], 'orange', label='Ping Latency (Transition)', linewidth=1.5)
ax1.plot(post_failure['timestamp'], post_failure['time_ms'], 'c-', label='Ping Latency (Backup Path)', linewidth=1.5)
ping_data['time_ms_smooth'] = ping_data['time_ms'].interpolate(method='linear')
ax1.plot(ping_data['timestamp'], ping_data['time_ms_smooth'], 'b--', alpha=0.5)

# Packet Loss Markers
loss_points = ping_data[ping_data['time_ms'].isna()]
if not loss_points.empty:
    ax1.scatter(loss_points['timestamp'], [7.5] * len(loss_points), color='red', marker='x', s=100, label='Packet Loss')
else:
    ax1.text(failure_time + 1, 5, 'No Packet Loss Observed', color='red', fontsize=10, bbox=dict(facecolor='white', alpha=0.8))

# Packet Loss Rate (left y-axis, scaled to fit with latency)
# Scale the packet loss rate to fit within the latency range (0–30 ms)
scaled_loss_rates = loss_rates * (30 / 100)  # Scale 0–100% to 0–30 ms range
ax1.fill_between(loss_timestamps, 0, scaled_loss_rates, color='red', alpha=0.2, label='Packet Loss Rate (%)')

ax1.set_ylabel('Latency (ms) / Scaled Packet Loss Rate', color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_title('Combined Ping Latency, Port Usage, and Packet Loss Rate Over Time')
ax1.grid(True, linestyle='--', alpha=0.7)
ax1.set_ylim(0, 30)
ax1.axvline(failure_time, color='r', linestyle='--', label='Link Failure')
ax1.annotate('Link Failure', xy=(failure_time, 15), xytext=(failure_time + 2, 20),
             arrowprops=dict(arrowstyle='->'), color='r')

# Highlight the transition window
ax1.axvspan(failure_time, reroute_time, color='yellow', alpha=0.2, label='Failure to Reroute Window')

# Add shaded regions for traffic flow
ax1.axvspan(0, failure_time, color='blue', alpha=0.05, label='Traffic on Primary Path (Port 3)')
ax1.axvspan(reroute_time, ping_data['timestamp'].max(), color='cyan', alpha=0.05, label='Traffic on Backup Path (Port 5)')

# Active Port Plot (right y-axis)
ax2 = ax1.twinx()
ax2.step(periodic_data['timestamp'], periodic_data['active_port'], 'g-', where='post', label='Active Port', linewidth=1.5)
ax2.set_ylabel('Active Port', color='g')
ax2.tick_params(axis='y', labelcolor='g')
ax2.set_ylim(2, 6)

# Annotate the port switch
switch_idx = periodic_data['active_port'].ne(periodic_data['active_port'].shift()).idxmax()
switch_time = periodic_data['timestamp'].iloc[switch_idx] if switch_idx < len(periodic_data) else failure_time
switch_port = periodic_data['active_port'].iloc[switch_idx] if switch_idx < len(periodic_data) else 5
ax2.annotate(f'Port Switch to {switch_port}', xy=(switch_time, switch_port), xytext=(switch_time + 2, switch_port + 0.5),
             arrowprops=dict(arrowstyle='->'), color='g')

# Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', frameon=True, edgecolor='black', fontsize=10)

ax1.set_xlabel('Time (seconds)')
plt.tight_layout(pad=2.0)
plt.savefig('combined_rerouting_plot.png')
plt.show()

# 2. Throughput Plot with Moving Average (unchanged)
fig, ax = plt.subplots(figsize=(14, 8))
if not iperf_data.empty and 'timestamp' in iperf_data.columns and 'throughput' in iperf_data.columns:
    pre_failure_iperf = iperf_data[iperf_data['timestamp'] < failure_time]
    post_failure_iperf = iperf_data[iperf_data['timestamp'] >= failure_time]
    ax.plot(pre_failure_iperf['timestamp'], pre_failure_iperf['throughput'], 'm-', label='Throughput (Pre-Failure)', linewidth=1.5)
    ax.plot(post_failure_iperf['timestamp'], post_failure_iperf['throughput'], 'purple', label='Throughput (Post-Failure)', linewidth=1.5)
    iperf_data['throughput_ma'] = iperf_data['throughput'].rolling(window=3, min_periods=1).mean()
    ax.plot(iperf_data['timestamp'], iperf_data['throughput_ma'], 'k--', label='Throughput (Moving Avg)', linewidth=1.5)
else:
    print("Warning: No valid iperf data to plot. Skipping throughput plot.")
ax.axvline(failure_time, color='r', linestyle='--', label='Link Failure')
ax.annotate('Link Failure', xy=(failure_time, 15), xytext=(failure_time + 2, 20),
            arrowprops=dict(arrowstyle='->'), color='r')
ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Throughput (Mbits/sec)')
ax.tick_params(axis='y')
ax.set_title('Throughput from h3 to h2 During Rerouting')
ax.grid(True, linestyle='--', alpha=0.7)
ax.set_ylim(0, 30)
ax.legend(loc='upper right', frameon=True, edgecolor='black', fontsize=10)
plt.tight_layout(pad=2.0)
plt.savefig('throughput_plot.png')
plt.show()

# 3. Enhanced Register Plot with Link Status and Backup Counter (unchanged)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
stages = ['Initial', 'Congestion (Pre-Failure)', 'Post-Failure']
link_status_s1 = [s1_initial['link_status'][3], s1_congestion['link_status'][3], s1_post['link_status'][3]]
backup_counters = [s1_initial['backup_counter'], s1_congestion['backup_counter'], s1_post['backup_counter']]

ax1.bar(stages, link_status_s1, color='purple', alpha=0.5, label='Primary Link Status (1=Up, 0=Down)')
ax1.set_xticks(range(len(stages)))
ax1.set_xticklabels(stages)
ax1.set_ylabel('Link Status')
ax1.set_title('s1 Primary Link Status Over Time')
ax1.legend()
for i, v in enumerate(link_status_s1):
    ax1.text(i, v + 0.05, str(v), ha='center', fontsize=10)

ax2.bar(stages, backup_counters, color='orange', alpha=0.5, label='Backup Counter')
ax2.set_xticks(range(len(stages)))
ax2.set_xticklabels(stages)
ax2.set_ylabel('Backup Counter')
ax2.set_title('s1 Backup Counter Over Time')
ax2.legend()
for i, v in enumerate(backup_counters):
    ax2.text(i, v + 0.5, str(v), ha='center', fontsize=10)

plt.tight_layout(pad=2.0)
plt.savefig('register_states_enhanced.png')
plt.show()

# 4. Cumulative Packet Loss Plot (unchanged)
fig, ax = plt.subplots(figsize=(14, 8))
ax.plot(cumulative_timestamps, cumulative_loss, 'r-', label='Cumulative Packet Loss', linewidth=1.5)
ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Cumulative Packet Loss (Packets)', color='r')
ax.tick_params(axis='y', labelcolor='r')
ax.set_title('Cumulative Packet Loss Over Time')
ax.grid(True, linestyle='--', alpha=0.7)
ax.axvline(failure_time, color='r', linestyle='--', label='Link Failure')
ax.annotate('Link Failure', xy=(failure_time, cumulative_loss[-1] / 2), xytext=(failure_time + 2, cumulative_loss[-1] / 2 + 1),
            arrowprops=dict(arrowstyle='->'), color='r')
ax.axvspan(failure_time, reroute_time, color='yellow', alpha=0.2, label='Failure to Reroute Window')
ax.legend(loc='upper left', frameon=True, edgecolor='black', fontsize=10)
plt.tight_layout(pad=2.0)
plt.savefig('cumulative_packet_loss_plot.png')
plt.show()